In [5]:
import sqlite3
from pymystem3 import Mystem
m = Mystem()
con = sqlite3.connect('FDB.db')
cur = con.cursor()

In [9]:
import pandas as pd
import re

# 0 Data preparation

Tags

In [10]:
with open('./raw_data/tags.csv','r') as f:
    tags = f.readlines()
    result = {t.split()[0]:t.split()[1] for t in tags}
print (result)

{'A': 'pos', 'ADV': 'pos', 'ADVPRO': 'pos', 'ANUM': 'pos', 'APRO': 'pos', 'COM': 'pos', 'CONJ': 'pos', 'INTJ': 'pos', 'NUM': 'pos', 'PART': 'pos', 'PR': 'pos', 'S': 'pos', 'SPRO': 'pos', 'V': 'pos', 'praes': 'tense', 'inpraes': 'tense', 'praet': 'tense', 'nom': 'case', 'gen': 'case', 'dat': 'case', 'acc': 'case', 'ins': 'case', 'abl': 'case', 'part': 'case', 'loc': 'case', 'voc': 'case', 'sg': 'num', 'pl': 'num', 'ger': 'vform', 'inf': 'vform', 'partcp': 'vform', 'indic': 'vform', 'imper': 'vform', 'brev': 'aform', 'plen': 'aform', 'poss': 'aform', 'supr': 'degree', 'comp': 'degree', '1p': 'pers', '2p': 'pers', '3p': 'pers', 'm': 'gender', 'f': 'gender', 'n': 'gender', 'ipf': 'asp', 'pf': 'asp', 'act': 'voice', 'pass': 'voice', 'tran': 'trans', 'intr': 'trans', 'parenth': 'type', 'praed': 'type', 'abbr': 'type', 'mf': 'type', 'awkw': 'form', 'dist': 'form', 'geo': 'sem', 'persn': 'sem', 'patrn': 'sem', 'famn': 'sem', 'obsc': 'style', 'inform': 'style', 'rare': 'style', 'obsol': 'style'

# 1 block

**Import informants**

In [123]:
db_1 = sqlite3.connect('./raw_data/ABM.sqlite')
db_1_cur = db_1.cursor()

In [124]:
db_1_cur.execute('SELECT * FROM tblInformators')
inf = db_1_cur.fetchall()

In [127]:
cur.executemany('INSERT INTO Informators (code, name, city, bio, old_id) VALUES (?,?,?,?,?)', inf)

In [128]:
con.commit()

**Import texts to database**

load data

In [102]:
db_1_cur.execute('SELECT * FROM tblCards')
texts = db_1_cur.fetchall()

In [29]:
for key, value in enumerate(texts[0]):
    print (key, value)

0 [Шёл разговор о пьющей женщине из Карпиловки, Г назвал её «Ма\ндыка».] [Ма\ндыка?] [Г:] Да. [Что это значит?] [Г, смеясь:] Ну, манды\ка — э\то… [А одновременно:] Коро\че её ёб… [Г:] Все. [А:] Всё село\. [Г:] Да, коро\че… [А:] И… и… и из на\шего, и коро\че там… [Г:] Все подря\д. [А:] Да. [А и Г одновременно:] Кому\ не лень. [А:] Буты\лку на стол, и… [Она пьёт, да?] [Г:] Ну, она\ по-чё\рному пьёт.
1 
2 
3 
4 XXIa
5 5 б
6 Брянская обл., Злынковский р-н, Спиридонова Буда
7 2017
8 А
9 Г
10 
11 
12 МЭГ
13 НВП
14 СЕ
15 ФЕЕ
16 1
17 
18 
19 
20 


In [116]:
texts2 = []

In [117]:
for row in texts:
    new_row = (
        'АБМ',
        row[16],
        row[6],
        row[0],
        row[7],
        restore_label(row[5]),
        row[4],
        ', '.join(i for i in row[8:12] if i != ''),
        ', '.join(i for i in row[12:16] if i != '') 
    )
    texts2.append(new_row)

In [121]:
cur.executemany('INSERT INTO Texts (leader, old_id, city, raw_text, year, question, list, informants, collectors)\
VALUES (?,?,?,?,?,?,?,?,?)', texts2)

In [122]:
con.commit()

2. change question labels

In [110]:
def restore_label(label):
    label = label.split(',')
    result = []
    #print(label)
    for i in label:
        q = re.findall('^([0-9]*)[^0-9a-zа-я]*?([ a-zа-я\.]*?)$', i.strip())[0]
        #print (q)
        result.append((q[0].strip(), q[1].strip()))
    for key, value in enumerate(result):
        if value[0] == '':
            result[key] = (result[key-1][0].strip(), value[1].strip())
    #print (result)
    return ', '.join(''.join(i) for i in result)

In [115]:
restore_label('15 а, 19 а, в, 30 а, 14 а, 25 а, б, 16 а')

'15а, 19а, 19в, 30а, 14а, 25а, 25б, 16а'

In [68]:
re.findall('^([0-9]*)[^0-9a-zа-я]*?([a-zа-я]*?)$',label[1].strip())

[('10', 'б')]

In [ ]:
data['Вопрос'] = restore_label(data['Вопрос'])

3. normalize text

In [132]:
from nltk.tokenize import sent_tokenize

In [133]:
def no_stress(text):
    text = text.replace('\\', '')
    return text

def split_sentences(text):
    text = text.replace('[', '\n[')
    text = re.sub('\n{2,}','\n')
    text = text.split('\n')
    text = [sent_tokenize(chunk) for chunk in text]
    return text

In [ ]:
def get_meta(code):
    cur.execute('SELECT id, gender, birthyear FROM Informants WHERE code=\"{}\"'.format(code))
    indiv = cur.fetchone()
    return {'id':indiv[0], 'gender':indiv[1], 'year':indiv[2]}

def get_col_id(code):
    cur.execute('SELECT id FROM Collectors WHERE code=\"{}\"'.format(code))
    indiv = cur.fetchone()
    return indiv[0]

def joint_meta(informants, collectors):
    informants = informants.split(',')
    result = {'inf':{}, 'col':{}}
    for i in informants:
        result['inf'][i.strip()] = get_meta(code)
    for i in collectors:
        result['col'][i.strip()] = get_col_id(code)

In [ ]:
def join_text(text, sentence):
    text = text
    for word in words:
            if 'wf' not in word:
                continue
            word['off_start'] = len(text)
            if word['wtype'] == 'word':
                text += word['wf'] + ' '
                word['off_end'] = len(text) - 1
            else:
                if word['wf'].startswith(('(', '[', '{', '<', '“')):
                    text += word['wf']
                    word['off_end'] = len(text)
                elif word['wf'].startswith((')', ']', '}', '>', '.', ',', '?', '!', '”', '…')):
                    if text.endswith(' '):
                        word['off_start'] -= 1
                        text = text[:-1]
                    text += word['wf'] + ' '
                    word['off_end'] = len(text) - 1
                else:
                    text += word['wf'] + ' '
                    word['off_end'] = len(text) - 1
        return text.strip()

In [ ]:
def ana(analysis):
    global tags
    result = {}
    for i in analysis:
        if i in tags:
            result[tags[i]] = i
    return result

In [ ]:
def tokens(parsed):
    result = []
    for i in parsed:
        if 'analysis' in i:
            r = {}
            r['wf'] = i['text']
            r['wtype'] = i['word']
            r['ana'] = []
            for var in i['analysis']:
                r['ana'].append(ana(var))
    return result

In [ ]:
def parse_sentence(sentence, meta):
    result = {'meta':{}, 'words':[]}
    if not re.match('^\[.*?\]$', sentence):
        sentence = sentence.split(']')
        sentence[0] = sentence[0]+']'
        parsed = tokens(m.analyze(sentence[1]))
    else:
        parsed = tokens(m.analyze(sentence))
    